In [1]:
import anndata
import torch
import stPlus

import squidpy as sq
import numpy as np
import scanpy as sc
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from transpa.eval_util import calc_corr
from transpa.util import compute_autocorr
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

seed = 10
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
spa_adata = sc.read_h5ad("../../data/ST/seqfish/seqfish_data.h5ad")
sc.pp.normalize_total(spa_adata)
sc.pp.log1p(spa_adata)

In [3]:
raw_spatial_df  = pd.DataFrame(spa_adata.X.toarray(), columns=spa_adata.var_names)

In [21]:
from sklearn.model_selection import KFold
pred_df_cross = pd.DataFrame(np.zeros(spa_adata.shape), columns=spa_adata.var_names)
X  = spa_adata.X.toarray()
for gene in tqdm(spa_adata.var_names):
    # rf = RandomForestRegressor(n_jobs=20)   
    kf = KFold(n_splits=3, shuffle=True) 
    train_X  = X[:, spa_adata.var_names != gene]
    for fd_train_idx, fd_test_idx in kf.split(train_X):
        rf = RandomForestRegressor(n_jobs=20) 
        rf.fit(train_X[fd_train_idx], y=X[fd_train_idx, spa_adata.var_names == gene].flatten())
        y = rf.predict(train_X[fd_test_idx])
        pred_df_cross.loc[fd_test_idx, gene] = y
    print(gene, " ", calc_corr(raw_spatial_df, pred_df_cross, [gene]))

corr_cross_bound_res = calc_corr(raw_spatial_df, pred_df_cross, pred_df_cross.columns)
print(np.median(corr_cross_bound_res))
corr_cross_bound_res = calc_corr(raw_spatial_df, pred_df_cross, pred_df_cross.columns, True)
print(np.median(corr_cross_bound_res))

  0%|          | 1/351 [05:05<29:42:08, 305.51s/it]

Abcc4   [0.12563713179754893]


  1%|          | 2/351 [10:49<31:47:51, 328.00s/it]

Acp5   [0.20416125218719391]


  1%|          | 3/351 [15:54<30:42:55, 317.75s/it]

...

100%|█████████▉| 350/351 [28:39:40<03:45, 225.76s/it]

Zfp57   [0.23580759200909351]


100%|██████████| 351/351 [28:44:40<00:00, 294.82s/it]

Zic3   [0.14978933700545327]


0.2274302109340357
0.3136927422721203
